<h1 align='center'>MSDE 692</h1>

[Function Definitions](#Function-Definitions)

[Data Cleaning](#Data-Cleaning)

In [202]:
import os
import pandas as pd
import neo4j
import numpy as np
import csv
import shlex

from scrapy.crawler import CrawlerProcess
from pprint import pprint
from deepdiff import DeepDiff
from functools import reduce
from datetime import datetime as dt
from csv import DictReader, writer

In [203]:
# https://www.youtube.com/watch?v=5Is-QdbKmEI

In [204]:
help(CrawlerProcess)

Help on class CrawlerProcess in module scrapy.crawler:

class CrawlerProcess(CrawlerRunner)
 |  CrawlerProcess(settings=None, install_root_handler=True)
 |  
 |  A class to run multiple scrapy crawlers in a process simultaneously.
 |  
 |  This class extends :class:`~scrapy.crawler.CrawlerRunner` by adding support
 |  for starting a :mod:`~twisted.internet.reactor` and handling shutdown
 |  signals, like the keyboard interrupt command Ctrl-C. It also configures
 |  top-level logging.
 |  
 |  This utility should be a better fit than
 |  :class:`~scrapy.crawler.CrawlerRunner` if you aren't running another
 |  :mod:`~twisted.internet.reactor` within your application.
 |  
 |  The CrawlerProcess object must be instantiated with a
 |  :class:`~scrapy.settings.Settings` object.
 |  
 |  :param install_root_handler: whether to install root logging handler
 |      (default: True)
 |  
 |  This class shouldn't be needed (since Scrapy is responsible of using it
 |  accordingly) unless writing s

## Function Definitions

In [359]:
def csv_reader(csv_file):
    final_list = []

    with open(csv_file, 'r') as f:
        input_list = csv.reader(f)
#         next(input_list)

        for row in input_list:
            final_list.append(row[0])
            
    return final_list

In [3]:
def people_pruner(my_file):
    my_list = []

    with open(my_file, 'r') as f:
        reader = DictReader(f, fieldnames='person')

        for row in reader:
            if [row['p']] not in my_list:
                my_list.append([row['p']])
            else:
                print(f"Duplicate person: {row['p']}")
    
    return my_list

In [90]:
def csv_writer(my_file, my_list, my_mode):

    with open(my_file, my_mode, newline='') as outfile:

        # using csv.writer method from CSV package
        csv_writer = writer(outfile)

        for row in my_list:
            csv_writer.writerows(row)

In [169]:
def csv_dict_writer(my_file, my_list, my_mode):
    with open(my_file, my_mode, newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['wiki'])
        
        for row in my_list:
            if row != 'wiki':
                writer.writerow({'wiki': row})

In [69]:
def csv_dict_appender(my_file, my_list):
    with open(my_file, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['wiki'])

        for row in my_list:
            writer.writerow({'wiki': row})

In [390]:
def add_new_wikis(adding_file, gaining_file, base_file):
    """
    Adds newly discovered wiki pages to an accumulative wiki pages file before deleting the previous base file
    used to extract the new wikis. The previous base file is then deleted and the adding_file is renamed to base_file
    for the next grabwikis spider run.
    """
    adding_list = csv_reader(adding_file)
    gaining_list = csv_reader(gaining_file)
    adding_set = set(adding_list)
    gaining_set = set(gaining_list)
    
    new_wikis = list(set.difference(adding_set, gaining_set))
    print(len(new_wikis))
    
    csv_dict_writer(gaining_file, new_wikis,'a')
    csv_dict_writer(base_file, new_wikis, 'w')
    
    os.remove(adding_file)

## Data Cleaning

### Create Difference CSV File for wikigrabber Spider Runs

In [391]:
add_new_wikis('wikigrabber/people.csv', 'wikigrabber/all_people.csv', 'wikigrabber/people_base.csv')

1012


### Explore people_data_temp.csv file

In [445]:
df = pd.read_csv('wikicrawler/people_data_temp.csv')
df

,schools,degrees,name,full_name,DOB,occupation,spouses,offspring
0,London School of Economics,"MSc,BSc",George Soros,György Schwartz,1930-08-12 00:00:00,"Investor, hedge fund manager, author, philanth...",Susan Weber (historian),"Jonathan Soros,Alexander Soros"
1,"Yale University,University of Wyoming,Universi...","MA,BA",Dick Cheney,Richard Bruce Cheney,1941-01-30 00:00:00,NaN,Lynne Cheney,"Liz Cheney,Mary Cheney"
2,University of Fribourg,NaN,Klaus Schwab,NaN,1938-03-30 00:00:00,World Economic Forum,Hilde Schwab,2
3,"DePauw University,Indiana University, Indianap...","BA,JD",Dan Quayle,James Danforth Quayle,1947-02-04 00:00:00,NaN,Marilyn Quayle,Ben Quayle
4,"Dartmouth College,University College, Oxford,Y...","MA,BA,JD",Robert Reich,Robert Bernard Reich,1946-06-24 00:00:00,NaN,Clare Dalton,Sam Reich
...,...,...,...,...,...,...,...,...
1310,"Harvard University,BA,MBA",NaN,Theodore Roosevelt V,Theodore Roosevelt V,1942-11-27 00:00:00,NaN,Constance Lane Rogers,Theodore Roosevelt VI
1311,Harvard University,"BA,LLB",Charles Francis Adams III,NaN,1866-08-02 00:00:00,NaN,index.php?title=Frances Adams&action=edit&redl...,Charles Francis Adams IV
1312,"Harvard University,Yale University","JD,AB",Kermit Roosevelt III,NaN,1971-07-14 00:00:00,NaN,NaN,NaN
1313,"Yale University,Princeton University,Universit...","MA,BA,JD",Bob Taft,Robert Alphonso Taft III,1942-01-08 00:00:00,NaN,NaN,Anna Taft


In [446]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   schools     842 non-null    object
 1   degrees     320 non-null    object
 2   name        1233 non-null   object
 3   full_name   581 non-null    object
 4   DOB         1225 non-null   object
 5   occupation  405 non-null    object
 6   spouses     698 non-null    object
 7   offspring   574 non-null    object
dtypes: object(8)
memory usage: 82.3+ KB


In [447]:
df_full = df.dropna(how='all')
df_full

,schools,degrees,name,full_name,DOB,occupation,spouses,offspring
0,London School of Economics,"MSc,BSc",George Soros,György Schwartz,1930-08-12 00:00:00,"Investor, hedge fund manager, author, philanth...",Susan Weber (historian),"Jonathan Soros,Alexander Soros"
1,"Yale University,University of Wyoming,Universi...","MA,BA",Dick Cheney,Richard Bruce Cheney,1941-01-30 00:00:00,NaN,Lynne Cheney,"Liz Cheney,Mary Cheney"
2,University of Fribourg,NaN,Klaus Schwab,NaN,1938-03-30 00:00:00,World Economic Forum,Hilde Schwab,2
3,"DePauw University,Indiana University, Indianap...","BA,JD",Dan Quayle,James Danforth Quayle,1947-02-04 00:00:00,NaN,Marilyn Quayle,Ben Quayle
4,"Dartmouth College,University College, Oxford,Y...","MA,BA,JD",Robert Reich,Robert Bernard Reich,1946-06-24 00:00:00,NaN,Clare Dalton,Sam Reich
...,...,...,...,...,...,...,...,...
1310,"Harvard University,BA,MBA",NaN,Theodore Roosevelt V,Theodore Roosevelt V,1942-11-27 00:00:00,NaN,Constance Lane Rogers,Theodore Roosevelt VI
1311,Harvard University,"BA,LLB",Charles Francis Adams III,NaN,1866-08-02 00:00:00,NaN,index.php?title=Frances Adams&action=edit&redl...,Charles Francis Adams IV
1312,"Harvard University,Yale University","JD,AB",Kermit Roosevelt III,NaN,1971-07-14 00:00:00,NaN,NaN,NaN
1313,"Yale University,Princeton University,Universit...","MA,BA,JD",Bob Taft,Robert Alphonso Taft III,1942-01-08 00:00:00,NaN,NaN,Anna Taft


In [458]:
# Drop all rows without a name
df_all_names = df_full.dropna(subset=['name'])
df_all_names                  

,schools,degrees,name,full_name,DOB,occupation,spouses,offspring
0,London School of Economics,"MSc,BSc",George Soros,György Schwartz,1930-08-12 00:00:00,"Investor, hedge fund manager, author, philanth...",Susan Weber (historian),"Jonathan Soros,Alexander Soros"
1,"Yale University,University of Wyoming,Universi...","MA,BA",Dick Cheney,Richard Bruce Cheney,1941-01-30 00:00:00,NaN,Lynne Cheney,"Liz Cheney,Mary Cheney"
2,University of Fribourg,NaN,Klaus Schwab,NaN,1938-03-30 00:00:00,World Economic Forum,Hilde Schwab,2
3,"DePauw University,Indiana University, Indianap...","BA,JD",Dan Quayle,James Danforth Quayle,1947-02-04 00:00:00,NaN,Marilyn Quayle,Ben Quayle
4,"Dartmouth College,University College, Oxford,Y...","MA,BA,JD",Robert Reich,Robert Bernard Reich,1946-06-24 00:00:00,NaN,Clare Dalton,Sam Reich
...,...,...,...,...,...,...,...,...
1310,"Harvard University,BA,MBA",NaN,Theodore Roosevelt V,Theodore Roosevelt V,1942-11-27 00:00:00,NaN,Constance Lane Rogers,Theodore Roosevelt VI
1311,Harvard University,"BA,LLB",Charles Francis Adams III,NaN,1866-08-02 00:00:00,NaN,index.php?title=Frances Adams&action=edit&redl...,Charles Francis Adams IV
1312,"Harvard University,Yale University","JD,AB",Kermit Roosevelt III,NaN,1971-07-14 00:00:00,NaN,NaN,NaN
1313,"Yale University,Princeton University,Universit...","MA,BA,JD",Bob Taft,Robert Alphonso Taft III,1942-01-08 00:00:00,NaN,NaN,Anna Taft


In [459]:
df_all_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   schools     835 non-null    object
 1   degrees     320 non-null    object
 2   name        1233 non-null   object
 3   full_name   581 non-null    object
 4   DOB         1163 non-null   object
 5   occupation  405 non-null    object
 6   spouses     697 non-null    object
 7   offspring   573 non-null    object
dtypes: object(8)
memory usage: 86.7+ KB


In [460]:
df_no_gaps = df_all_names.drop_duplicates()
df_no_gaps

,schools,degrees,name,full_name,DOB,occupation,spouses,offspring
0,London School of Economics,"MSc,BSc",George Soros,György Schwartz,1930-08-12 00:00:00,"Investor, hedge fund manager, author, philanth...",Susan Weber (historian),"Jonathan Soros,Alexander Soros"
1,"Yale University,University of Wyoming,Universi...","MA,BA",Dick Cheney,Richard Bruce Cheney,1941-01-30 00:00:00,NaN,Lynne Cheney,"Liz Cheney,Mary Cheney"
2,University of Fribourg,NaN,Klaus Schwab,NaN,1938-03-30 00:00:00,World Economic Forum,Hilde Schwab,2
3,"DePauw University,Indiana University, Indianap...","BA,JD",Dan Quayle,James Danforth Quayle,1947-02-04 00:00:00,NaN,Marilyn Quayle,Ben Quayle
4,"Dartmouth College,University College, Oxford,Y...","MA,BA,JD",Robert Reich,Robert Bernard Reich,1946-06-24 00:00:00,NaN,Clare Dalton,Sam Reich
...,...,...,...,...,...,...,...,...
1310,"Harvard University,BA,MBA",NaN,Theodore Roosevelt V,Theodore Roosevelt V,1942-11-27 00:00:00,NaN,Constance Lane Rogers,Theodore Roosevelt VI
1311,Harvard University,"BA,LLB",Charles Francis Adams III,NaN,1866-08-02 00:00:00,NaN,index.php?title=Frances Adams&action=edit&redl...,Charles Francis Adams IV
1312,"Harvard University,Yale University","JD,AB",Kermit Roosevelt III,NaN,1971-07-14 00:00:00,NaN,NaN,NaN
1313,"Yale University,Princeton University,Universit...","MA,BA,JD",Bob Taft,Robert Alphonso Taft III,1942-01-08 00:00:00,NaN,NaN,Anna Taft


In [461]:
df_no_gaps.to_csv('wikicrawler/people_data.csv')

In [462]:
df_degrees = df_no_gaps.dropna(subset=['schools', 'degrees'])

In [463]:
# Rearrange columns and sort on name
df_degrees = df_degrees[['name', 'full_name', 'DOB', 'occupation', 'schools', 'degrees', 'spouses', 'offspring']]
df_degrees.sort_values(by='name', ignore_index=True, inplace=True)
df_degrees

,name,full_name,DOB,occupation,schools,degrees,spouses,offspring
0,A. Clayton Spencer,Ava Clayton Spencer,1954-12-15 00:00:00,NaN,"Williams College,University of Oxford,Harvard ...","MA,BA,JD",Ash Carter,2
1,Abdul El-Sayed,Abdulrahman Mohamed El-Sayed,1984-10-31 00:00:00,NaN,"University of Michigan,Oriel College, Oxford,C...","MA,PhD,MD,BS",Sarah Jukaku,NaN
2,Alan Bersin,NaN,1946-10-15 00:00:00,NaN,"Harvard University,Yale University","BA,JD",Lisa Foster,NaN
3,Alan Chester Valentine,NaN,1901-02-23 00:00:00,NaN,"Swarthmore College,University of Pennsylvania,...","University of Pennsylvania,Balliol College, Ox...",Lucia Garrison Norton,Annie Laurie Buffinton
4,Alan Greenspan,NaN,1926-03-06 00:00:00,NaN,New York University,"PhD,MA,BA",Andrea Mitchell,NaN
...,...,...,...,...,...,...,...,...
308,William McAdoo,William Gibbs McAdoo Jr.,1863-10-31 00:00:00,NaN,"University of Tennessee, Knoxville",BA,Eleanor Wilson McAdoo,9
309,William McRae,William Allan McRae Jr.,1909-09-25 00:00:00,NaN,"University of Florida,University of Oxford,Fre...","B.A.,J.D.,B.Litt.,A.B.,B.A.,J.D.,B.Litt.,A.B.",NaN,NaN
310,William Taft,NaN,1945-09-13 00:00:00,NaN,"Yale University,Harvard University","BA,JD",Julia V. Taft,3
311,Wilson Elkins,Wilson Homer Elkins,1908-07-09 00:00:00,NaN,"University of Texas, Austin","MA,BA",Dorothy Blackburn,2


In [454]:
df_degrees.set_index('name', drop=True, inplace=True)
df_degrees

,full_name,DOB,occupation,schools,degrees,spouses,offspring
name,,,,,,,
A. Clayton Spencer,Ava Clayton Spencer,1954-12-15 00:00:00,NaN,"Williams College,University of Oxford,Harvard ...","MA,BA,JD",Ash Carter,2
Abdul El-Sayed,Abdulrahman Mohamed El-Sayed,1984-10-31 00:00:00,NaN,"University of Michigan,Oriel College, Oxford,C...","MA,PhD,MD,BS",Sarah Jukaku,NaN
Alan Bersin,NaN,1946-10-15 00:00:00,NaN,"Harvard University,Yale University","BA,JD",Lisa Foster,NaN
Alan Chester Valentine,NaN,1901-02-23 00:00:00,NaN,"Swarthmore College,University of Pennsylvania,...","University of Pennsylvania,Balliol College, Ox...",Lucia Garrison Norton,Annie Laurie Buffinton
Alan Greenspan,NaN,1926-03-06 00:00:00,NaN,New York University,"PhD,MA,BA",Andrea Mitchell,NaN
...,...,...,...,...,...,...,...
William McAdoo,William Gibbs McAdoo Jr.,1863-10-31 00:00:00,NaN,"University of Tennessee, Knoxville",BA,Eleanor Wilson McAdoo,9
William McRae,William Allan McRae Jr.,1909-09-25 00:00:00,NaN,"University of Florida,University of Oxford,Fre...","B.A.,J.D.,B.Litt.,A.B.,B.A.,J.D.,B.Litt.,A.B.",NaN,NaN
William Taft,NaN,1945-09-13 00:00:00,NaN,"Yale University,Harvard University","BA,JD",Julia V. Taft,3


In [455]:
# df_degrees = df_degrees.replace({np.nan:None})

In [456]:
def date_converter(x):
    if x:
        if '-' in x:
            updated = dt.strptime(str(x), '%Y-%m-%d %H:%M:%S')
        elif '/' in x:
            updated = dt.strptime(str(x), '%m/%d/%Y %H:%M:%S')
        else:
            return None
        print(updated)
        return updated.strftme('%Y-%m-%d')        
    else:
        return None

In [457]:
# df_degrees['DOB'] = np.vectorize(date_converter)(df_degrees['DOB'])
df_degrees['DOB'] = pd.to_datetime(df_degrees.DOB)
df_degrees['DOB'] = df_degrees['DOB'].dt.strftime('%Y-%m-%d')
df_degrees

ValueError: ('Unknown string format:', '1989 or 1990 (age\xa031–32)')

In [ ]:
df_degrees.to_csv('wikicrawler/people_data_full.csv')

## Rhodes Scholars

In [ ]:
rhodes_df = pd.read_csv('rhodescholars/rhodes_data.csv')
rhodes_df

In [ ]:
df_occupations = rhodes_df.merge(df_no_gaps, on='name', how='outer')
df_occupations

In [414]:
df_occupations.to_csv('wikicrawler/merged_occupations.csv')